https://stackoverflow.com/questions/53252181/python-worker-failed-to-connect-back

Schema: https://stackoverflow.com/questions/45276427/unable-connect-to-docker-container-outside-docker-host

Config: https://stackoverflow.com/questions/72113678/hdfs-config-for-pyspark

IP docker: https://stackoverflow.com/questions/61432807/hadoop-spark-there-are-1-datanodes-running-and-1-nodes-are-excluded-in-th

Hosts windows: https://fixergeek.com/fixes/how-to-assign-hostname-to-ip-address-in-windows#:~:text=Press%20the%20Windows%20button%20%2B,key.%20IP%20configuration%20of%20Windows

Windows 11: https://www.sysgeeker.com/blog/how-to-add-to-host-file-windows-11.html

C:\Windows\System32\drivers\etc\hosts

hdfs dfsadmin -report

https://ruslanmv.com/blog/How-to-install-Hadoop-on-Windows

In [2]:
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch
from pyspark.sql.types import *
import findspark
findspark.init()

In [3]:
client = Elasticsearch("http://localhost:9200")

client.ping()

True

In [4]:
sparkSession = SparkSession.builder.appName("csv4").config("spark.hadoop.dfs.client.use.datanode.hostname", "true").config("spark.hadoop.dfs.datanode.use.datanode.hostname", "true").getOrCreate()

In [12]:
sparkSession2 = SparkSession.builder.remote('sc://localhost:7077').getOrCreate()

c:\users\sergey.astakhov\desktop\bmstu-m2-db-cw\venv\lib\site-packages\pyspark\sql\connect\session.py:185: UserWarning: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv6:%5B::1%5D:7077: End of TCP stream"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-02-24T16:26:38.9291319+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv6:%5B::1%5D:7077: End of TCP stream"}"
>
  warnings.warn(str(e))
c:\users\sergey.astakhov\desktop\bmstu-m2-db-cw\venv\lib\site-packages\pyspark\sql\connect\session.py:185: UserWarning: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:7077: End of TCP stream"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-02-24T16:36:51.6222518+00:00

In [ ]:
sparkSession3 = SparkSession.builder.remote('sc://localhost:7077').getOrCreate()

In [6]:
sparkSession

In [ ]:
sparkSession2

In [7]:
searchBody = {
  "size": 9999,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="master", body=searchBody)
masters = result['hits']['hits']
result = client.search(index="order", body=searchBody)
orders = result['hits']['hits']

In [8]:
masters[:1]

[{'_index': 'master',
  '_id': '618529',
  '_score': 1.0,
  '_source': {'master_id': 618529,
   'master_desc': 'Аполлон Харитонович Афанасьев, Стаж Работы: 5 Л./Г..',
   'master_feedbacks': ['медлительный, аккуратный.',
    'ворчливый, медлительный.']}}]

In [9]:
orders[:1]

[{'_index': 'order',
  '_id': '939218',
  '_score': 1.0,
  '_source': {'order_id': 939218,
   'order_date': '2024-01-27',
   'order_customer_id': 290404,
   'order_customer_desc': 'Имя: Жанна Владиславовна Семенова; должник, постоянный.',
   'order_details_desc': 'тяжелый, объемный.',
   'order_due_date': '2024-03-24',
   'order_fact_completion_date': '2024-04-11',
   'order_parts': ['корпус'],
   'repair_types': ['замена компонентов'],
   'order_price': 8642.84646,
   'order_master_id': 96520}}]

In [10]:
MasterSchema = StructType([
    StructField("master_id", IntegerType(), False),
    StructField("master_desc", StringType(), False),
    StructField("master_feedbacks", StringType(), False)
])
CustomerSchema = StructType([
    StructField("order_customer_id", IntegerType(), False),
    StructField("order_customer_desc", StringType(), False)
])
OrderSchema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("order_customer_id", IntegerType(), False),
    # StructField("order_customer_desc", StringType(), False),
    StructField("order_details_desc", StringType(), False),
    StructField("order_due_date", StringType(), False),
    StructField("order_fact_completion_date", StringType(), False),
    StructField("order_parts", StringType(), False),
    StructField("repair_types", StringType(), False),
    StructField("order_price", FloatType(), False),
    StructField("order_master_id", IntegerType(), False)
])

In [11]:
MasterTable = []
CustomerTable = []
OrderTable = []

In [12]:
CustomerSet = set()

In [13]:
for master in masters:
    MasterTable.append((
        master['_source']['master_id'],
        master['_source']['master_desc'],
        str(master['_source']['master_feedbacks'])
    ))
for order in orders:
    OrderTable.append((
        order['_source']['order_id'],
        order['_source']['order_date'],
        order['_source']['order_customer_id'],
        order['_source']['order_details_desc'],
        order['_source']['order_due_date'],
        order['_source']['order_fact_completion_date'],
        str(order['_source']['order_parts']),
        str(order['_source']['repair_types']),
        order['_source']['order_price'],
        order['_source']['order_master_id']
    ))
    if not order['_source']['order_id'] in CustomerSet:
        CustomerSet.add(order['_source']['order_id'])
        CustomerTable.append((
            order['_source']['order_customer_id'],
            order['_source']['order_customer_desc']
        ))

In [14]:
MasterDF = sparkSession.createDataFrame(MasterTable, MasterSchema)
OrderDF = sparkSession.createDataFrame(OrderTable, OrderSchema)
CustomerDF = sparkSession.createDataFrame(CustomerTable,CustomerSchema)

In [15]:
MasterDF.show()

+---------+--------------------+--------------------+
|master_id|         master_desc|    master_feedbacks|
+---------+--------------------+--------------------+
|   618529|Аполлон Харитонов...|['медлительный, а...|
|   791839|Август Дорофеевич...|['медлительный, в...|
|   800991|Сысоев Селиверст ...|['аккуратный, вор...|
|   979439|Никанор Всеволодо...|['аккуратный, мед...|
|   245954|Леон Власович Хох...|['аккуратный, мед...|
|    96520|Давыдова Александ...|['аккуратный, мед...|
|   377918|Дьячков Евстигней...|['ворчливый, акку...|
|   185439|Никонова Нинель Б...|['медлительный, в...|
|   717684|Михеев Тарас Тейм...|['ворчливый, медл...|
|   827062|Никонова Наталья ...|['аккуратный, вор...|
|   303644|Гордеева Виктория...|['ворчливый, медл...|
|   962470|Мокей Матвеевич Е...|['медлительный, в...|
|   319301|Громова Жанна Бог...|['ворчливый, акку...|
|   325551|Фадеев Демид Адам...|['медлительный, в...|
|   140834|Анна Валентиновна...|['медлительный, в...|
|   678593|Беляева Ульяна Аф

In [16]:
OrderDF.show()

+--------+----------+-----------------+------------------+--------------+--------------------------+--------------------+--------------------+-----------+---------------+
|order_id|order_date|order_customer_id|order_details_desc|order_due_date|order_fact_completion_date|         order_parts|        repair_types|order_price|order_master_id|
+--------+----------+-----------------+------------------+--------------+--------------------------+--------------------+--------------------+-----------+---------------+
|  939218|2024-01-27|           290404|тяжелый, объемный.|    2024-03-24|                2024-04-11|          ['корпус']|['замена компонен...|   8642.847|          96520|
|  416896|2024-01-26|           344599|объемный, тяжелый.|    2024-04-21|                2024-04-20|['аккумулятор', '...|['замена компонен...|  7869.5522|         717684|
|  696901|2024-02-08|           500287| ценный, объемный.|    2024-04-01|                2024-04-13|['корпус', 'аккум...|['мелкие детали',...|   

In [17]:
CustomerDF.show()

+-----------------+--------------------+
|order_customer_id| order_customer_desc|
+-----------------+--------------------+
|           290404|Имя: Жанна Владис...|
|           344599|Имя: Гаврила Авер...|
|           500287|Имя: Орехова Нине...|
|           563995|Имя: Котова Октяб...|
|           524235|Имя: Гаврилов Гео...|
|           682922|Имя: Дроздов Аник...|
|           185734|Имя: Громов Евгра...|
|           829204|Имя: Жанна Роберт...|
|           824162|Имя: Нинель Олего...|
|           809737|Имя: Жданов Аниси...|
|           258409|Имя: тов. Петров ...|
|           410014|Имя: Юрий Фролови...|
|           259626|Имя: Алексеев Кор...|
|           684839|Имя: Смирнова Дар...|
|           318498|Имя: Самойлова Но...|
|           999257|Имя: Онуфрий Герм...|
|           339737|Имя: тов. Елисеев...|
|           549823|Имя: Варвара Серг...|
|           416915|Имя: Агафья Богда...|
|           172851|Имя: Логинова Оль...|
+-----------------+--------------------+
only showing top

In [18]:
# MasterDF.write.csv(path='file:///C:/folder1/masters.csv',mode='overwrite', header=True)

In [146]:
# OrderDF.write.csv(path='hdfs://localhost:9010/orders.csv',mode='overwrite', header=True)

In [19]:
# CustomerDF.write.csv(path='hdfs://localhost:9010/customer.csv',mode='overwrite', header=True)
CustomerDF.write.csv(path='hdfs://namenode:9000/customer.csv',mode='overwrite', header=True)

Py4JJavaError: An error occurred while calling o76.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 7.0 failed 1 times, most recent failure: Lost task 2.0 in stage 7.0 (TID 16) (NB3448.inside.glowbyte.com executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://namenode:9000/customer.csv.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.comput

In [11]:
sparkSession.stop()